Use llamaindex agent: https://microsoft.github.io/autogen/docs/notebooks/agentchat_group_chat_with_llamaindex_agents/

In [1]:
import sys

sys.path.append('/Users/lzchen/PycharmProjects/llamaindex-sample/src')

In [3]:
import os
import autogen
from web_scraping_agent.utils.config import settings

### llm

In [4]:
config_list = [{"model": settings.AZURE_OPENAI_GPT4O_MODEL, "api_key": settings.AZURE_OPENAI_API_KEY}]


In [6]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings

llm = AzureOpenAI(
    azure_deployment=settings.AZURE_OPENAI_GPT4O_MODEL,
    temperature=0.1,
    azure_endpoint=settings.AZURE_OPENAI_ENDPOINT,
    api_key=settings.AZURE_OPENAI_API_KEY,
    api_version=settings.AZURE_OPENAI_API_VERSION,
)

Settings.llm = llm

### Agents

In [7]:
from llama_index.core.agent import ReActAgent
from web_scraping_agent.tools.web_parsing import li_scraper_tool, li_url_validation_tool

web_searcher = ReActAgent.from_tools(tools=[li_scraper_tool, li_url_validation_tool], llm=llm, max_iterations=10,
                                     verbose=True)


In [8]:
from autogen.agentchat.contrib.llamaindex_conversable_agent import LLamaIndexConversableAgent

llm_config = {
    "temperature": 0.1,
    "config_list": config_list,
}

research_assistant_sys_msg = """
You help user to get the main information about a company from provided website, including
 their goal, detailed summary of the company, established time, and the key members (Founder, CEO, COO, CTO etc.) of the company. Return your result as a JSON with four keys:
 company_goal, company_summary, established_time, and key_members. key_members contains a list of
 dictionaries with keys 'name', 'role' and 'linkedin_page'. If the linkedIn page is not directly available on the given page content, it should be an empty string, do not try to come up with made up linkedin profile.
"""
research_assistant_desc = """
This agent helps investment firm to gather main information about a company of interest by scraping and extracting information form the provided company website. 
"""
research_assistant = LLamaIndexConversableAgent(
    "research_assistant",
    llama_index_agent=web_searcher,
    system_message=research_assistant_sys_msg,
    description=research_assistant_desc,
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="ALWAYS",
    code_execution_config=False,
)

In [9]:
groupchat = autogen.GroupChat(
    agents=[research_assistant, user_proxy],
    messages=[],
    max_round=500,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

[autogen.oai.client: 07-13 16:52:00] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[2024-07-13 16:52:00 - autogen.oai.client:164 - WARNING] The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [10]:
chat_result = user_proxy.initiate_chat(
    manager,
    message="Should I invest in this company https://vespa.ai/",
)

Admin (to chat_manager):

Should I invest in this company https://vespa.ai/

--------------------------------------------------------------------------------

Next speaker: research_assistant


>>>>>>>> USING AUTO REPLY...
Thought: The current language of the user is English. I need to use a tool to help me gather information about the company.
Action: crawl_web_page
Action Input: {'page_url': 'https://vespa.ai/'}
Observation: [![Vespa logo](https://vespa.ai/assets/logo.png)](/)

[](https://search.vespa.ai/)

[Get Started](https://cloud.vespa.ai/en/getting-started)
 [Features](/features)

Solutions

[Vespa Executive Overview](/executive-overview.html)
 [Unlocking eCommerce Growth](/unlocking-ecommerce-growth.html)

Documentation

[Vespa Overview Deck](https://docs.google.com/presentation/d/1TO_QX451CUvLzI3rVMcjB0qQXutUaoF6rTVfQLrfvPg)
 [Vespa Documentation](https://docs.vespa.ai/)
 [Video Archive](/resources)
 [Vespa and ElasticSearch / Solr](/vespa-elastic-solr)

[Blog](https://blog.v

### Tasks